In [1]:
import datetime as dt
import lakeapi
import cudf
import pandas as pd
import numpy as np

/home/andras/miniconda3/envs/rapids-25.02/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # ─── 1. PARAMÉTEREK ────────────────────────────────────────────────────────────
start_date = dt.datetime(2024, 9, 1)
end_date   = dt.datetime(2024, 9, 15)
symbol     = "BTC-USDT"
exchange   = "BINANCE"

# df_book = lakeapi.load_data(
#     table     = "book",
#     start     = start_date,
#     end       = end_date,
#     symbols   = [symbol],
#     exchanges = [exchange],
# )

# ─── 3. DINAMIKUS FÁJLNÉV ──────────────────────────────────────────────────────
file_name = (
    f"data/book_{symbol.lower().replace('-', '_')}_"
    f"{start_date:%Y%m%d}_{end_date:%Y%m%d}.parquet"
)

# ─── 4. MENTÉS PARQUET-BE cudf használatával ─────────────────────────────────────────
# df_book.to_parquet(
#     file_name,
#     engine="pyarrow",    # cuDF támogatja a pyarrow és fastparquet engine-t
#     compression="snappy",
# )

# print(f"Mentés kész: {file_name}")


In [3]:
def load_book_parquet(
    symbol: str,
    start_date: dt.date | dt.datetime | str,
    end_date: dt.date | dt.datetime | str,
    data_dir: str = "./data",
    use_gpu: bool = True,
) -> cudf.DataFrame | pd.DataFrame:
    """
    Beolvassa a ./data mappában lévő, symbol és date alapján elnevezett
    parquet file-t, és visszaadja DataFrame-ként.
    
    Parameters:
        symbol: A kereskedett eszköz azonosítója
        start_date: Kezdő dátum
        end_date: Záró dátum
        data_dir: Az adatfájlokat tartalmazó mappa elérési útja
        use_gpu: Ha True, akkor cuDF DataFrame-et ad vissza (GPU-gyorsított),
                ha False, akkor pandas DataFrame-et
    
    Returns:
        cudf.DataFrame vagy pd.DataFrame az use_gpu paraméter értékétől függően
    """
    def _to_date(x):
        if isinstance(x, str):
            return dt.datetime.strptime(x, "%Y-%m-%d").date()
        if isinstance(x, dt.datetime):
            return x.date()
        return x

    sd = _to_date(start_date)
    ed = _to_date(end_date)

    fname = (
        f"{data_dir}/book_"
        f"{symbol.lower().replace('-', '_')}_"
        f"{sd:%Y%m%d}_{ed:%Y%m%d}.parquet"
    )

    if use_gpu:
        # cuDF natív, GPU-s olvasó engine
        return cudf.read_parquet(fname, engine="cudf")
    else:
        # Pandas CPU-alapú olvasás
        return pd.read_parquet(fname)

In [4]:
# Tegyük fel, hogy korábban mentetted:
#   ./data/book_btc_usdt_20250301_20250305.parquet

df_book = load_book_parquet(
    symbol="BTC-USDT",
    start_date=start_date,
    end_date=end_date,
    use_gpu=True
)

print(type(df_book))       # use_gpu paramétertől függően cudf.DataFrame vagy pd df
print(f' sor: {len(df_book)}, \n {df_book.head()}')


<class 'cudf.core.dataframe.DataFrame'>
 sor: 8584947, 
                     origin_time                 received_time  \
0 2024-09-01 00:00:00.013999872 2024-09-01 00:00:00.031965952   
1 2024-09-01 00:00:01.313999872 2024-09-01 00:00:01.661474560   
2 2024-09-01 00:00:02.114000128 2024-09-01 00:00:02.292552448   
3 2024-09-01 00:00:02.214999808 2024-09-01 00:00:02.381574144   
4 2024-09-01 00:00:02.313999872 2024-09-01 00:00:02.499562240   

   sequence_number  bid_0_price  bid_0_size  bid_1_price  bid_1_size  \
0      51053096868     58973.99     1.36042     58973.98     0.00357   
1      51053097071     58973.99     1.31635     58973.98     0.00357   
2      51053097164     58973.99     1.33168     58973.98     0.00357   
3      51053097175     58973.99     1.33328     58973.98     0.00357   
4      51053097183     58973.99     1.33328     58973.98     0.00357   

   bid_2_price  bid_2_size  bid_3_price  ...  ask_16_price  ask_16_size  \
0     58973.41     0.00009     58973.40  ...

In [5]:
# Dátum ellenőrzés
min_origin   = df_book['origin_time'].min()
max_origin   = df_book['origin_time'].max()
min_received = df_book['received_time'].min()
max_received = df_book['received_time'].max()

# Duration másodpercben
duration_origin_s   = (max_origin - min_origin) / np.timedelta64(1, "s")
duration_received_s = (max_received - min_received) / np.timedelta64(1, "s")

print(f"""
Requested time range:
    Start: {start_date}
    End:   {end_date}

Actual data range:
    Origin time:
        Min: {min_origin}
        Max: {max_origin}
        Duration: {duration_origin_s:.0f} s
    
    Received time:
        Min: {min_received}
        Max: {max_received}
        Duration: {duration_received_s:.0f} s

Number of records: {len(df_book):,}
Average records per second: {len(df_book) / duration_received_s:,.2f}
""")

# Gaps vizsgálata
df_book['next_origin_time'] = df_book['origin_time'].shift(-1)
df_book['time_gap'] = df_book['next_origin_time'] - df_book['origin_time']
max_gap = df_book['time_gap'].max()

max_gap_s = max_gap / np.timedelta64(1, "s")
if max_gap_s > 1:
    print(f"\nWarning: Found gaps in data!")
    print(f"Largest gap: {max_gap_s:.2f} seconds")
    gap_location = df_book.loc[df_book['time_gap'] == max_gap, 'origin_time'].iloc[0]
    print(f"Largest gap starts at: {gap_location}")



Requested time range:
    Start: 2024-09-01 00:00:00
    End:   2024-09-15 00:00:00

Actual data range:
    Origin time:
        Min: 1970-01-01T00:00:00.000000000
        Max: 2024-09-14T23:59:59.914000128
        Duration: 1726358400 s
    
    Received time:
        Min: 2024-09-01T00:00:00.031965952
        Max: 2024-09-14T23:59:59.916381696
        Duration: 1209600 s

Number of records: 8,584,947
Average records per second: 7.10


Largest gap: 1726200248.31 seconds
Largest gap starts at: origin_time   1970-01-01
Name: 7429460, dtype: datetime64[ns]


In [6]:
# 1) received_time oszlop pandasra hozása (cuDF → pandas),
#    de ha már pandas, hagyjuk változatlanul
recv_raw = (
    df_book["received_time"].to_pandas()
    if hasattr(df_book["received_time"], "to_pandas")        # cuDF eset
    else df_book["received_time"]                            # már pandas
)


# ──────────────────────────────────────────────────────────────
# 2) formátum-ellenőrzés  ➜  invalid_df
# ──────────────────────────────────────────────────────────────
recv_dt = pd.to_datetime(recv_raw, errors="coerce")        # ami hibás, NaT lesz
invalid_mask = recv_dt.isna()

invalid_df = pd.DataFrame({
    "row_id": np.where(invalid_mask)[0],
    "invalid_value": recv_raw[invalid_mask].values
})
if not invalid_df.empty:
    print(f"⚠️  Invalid timestamp format rows: {len(invalid_df):,}")
    print(invalid_df.head())
else:
    print("✓ Minden timestamp parse-olható ISO formátumra.")

# csak a helyes dátumokkal megyünk tovább
recv = recv_dt.dropna()

# ──────────────────────────────────────────────────────────────
# 3) alap statisztikák az érvényes dátumokra
# ──────────────────────────────────────────────────────────────
min_recv, max_recv = recv.min(), recv.max()
duration_s = (max_recv - min_recv) / np.timedelta64(1, "s")

print(f"""
Actual received_time range:
    Min: {min_recv}
    Max: {max_recv}
    Duration: {duration_s:.0f} s

Number of valid records: {len(recv):,}
Average records per second: {len(recv) / duration_s:,.2f}
""")

# ──────────────────────────────────────────────────────────────
# 4) hiányzó EGÉSZ másodpercek  ➜  missing_df
# ──────────────────────────────────────────────────────────────
secs = recv.dt.floor("s")
unique_secs = secs.drop_duplicates().sort_values()

full_range = pd.date_range(unique_secs.iloc[0],
                           unique_secs.iloc[-1],
                           freq="S")
missing_secs = full_range.difference(unique_secs)

missing_df = pd.DataFrame({"missing_second": missing_secs})
print(f"Hiányzó egész másodpercek: {len(missing_df):,}")

# ──────────────────────────────────────────────────────────────
# 5) legnagyobb valós gap  (akár <1 s, akár >1 s)
# ──────────────────────────────────────────────────────────────
gaps = recv.sort_values().diff().dropna()
max_gap = gaps.max()
max_gap_s = max_gap / np.timedelta64(1, "s")

if max_gap_s > 1:
    gap_start = recv.sort_values().iloc[gaps.argmax()]
    print(f"⚠️  Largest gap: {max_gap_s:.2f} seconds   (starts at {gap_start})")
else:
    print("✓ Nem találtunk 1 másodpercnél nagyobb valós gapet.")

# ──  opcionálisan: missing_df és invalid_df elmentése ─────────
# missing_df.to_parquet("data/missing_seconds.parquet", index=False)
# invalid_df.to_parquet("data/invalid_timestamps.parquet", index=False)


✓ Minden timestamp parse-olható ISO formátumra.

Actual received_time range:
    Min: 2024-09-01 00:00:00.031965952
    Max: 2024-09-14 23:59:59.916381696
    Duration: 1209600 s

Number of valid records: 8,584,947
Average records per second: 7.10



/tmp/ipykernel_376984/1199458193.py:51: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  full_range = pd.date_range(unique_secs.iloc[0],


Hiányzó egész másodpercek: 73
⚠️  Largest gap: 9.65 seconds   (starts at 2024-09-11 02:54:00.916120576)


### Napokra bontott

In [7]:
# import datetime as dt
# import numpy as np
# import pandas as pd

# # ──────────────────────────────────────────────────────────────
# # 1)  fair  &  mid  minden snapshotra   (VAMP, nincs aggregálás)
# # ──────────────────────────────────────────────────────────────
# def prepare_fair_mid(
#     df: pd.DataFrame,
#     N: int = 20,
#     time_col: str = "received_time",
# ) -> pd.DataFrame:
#     """20‐szintű VAMP-fair és mid-price kiszámítása, soronként."""
#     bid_p = df[[f"bid_{i}_price" for i in range(N)]].to_numpy()
#     bid_s = df[[f"bid_{i}_size"  for i in range(N)]].to_numpy()
#     ask_p = df[[f"ask_{i}_price" for i in range(N)]].to_numpy()
#     ask_s = df[[f"ask_{i}_size"  for i in range(N)]].to_numpy()

#     num = (bid_p * bid_s).sum(axis=1) + (ask_p * ask_s).sum(axis=1)
#     den =  bid_s.sum(axis=1)          +  ask_s.sum(axis=1)
#     den[den == 0] = np.nan                          # ne osztjunk 0-val
#     vamp = num / den

#     out = pd.DataFrame({
#         "time": pd.to_datetime(df[time_col], errors="coerce"),
#         "fair": vamp,
#         "mid":  (df["bid_0_price"] + df["ask_0_price"]) / 2,
#     })
#     return out


# # ──────────────────────────────────────────────────────────────
# # 2)  gördülő-szórású PnL
# # ──────────────────────────────────────────────────────────────
# def trading_pnl_realtime(
#     fm: pd.DataFrame,
#     window: str = "24h",
#     return_trade_log: bool = False,
# ):
#     df = fm.copy()
#     orig = len(df)

#     #   a) timestamp-hibák
#     bad_time = df["time"].isna().sum()
#     if bad_time:
#         print(f"⚠️  Invalid timestamp → NaT: {bad_time:,}")
#     df = df.dropna(subset=["time"])

#     #   b) NaN fair / mid
#     bad_price = df["fair"].isna() | df["mid"].isna()
#     if bad_price.any():
#         n_bad = bad_price.sum()
#         print(f"⚠️  NaN fair/mid: {n_bad:,}")
#         df = df[~bad_price]

#     print(f"✓ Valid rows for PnL: {len(df):,} / {orig:,}")

#     if len(df) == 0:                          # nincs adat → early exit
#         return 1.0, 0, pd.DataFrame() if return_trade_log else None

#     # --- 3) signal + gördülő σ  (csak ha TELJES ablak >= 24h) --------------
#     df = df.sort_values("time")
#     df["signal"] = df["fair"] - df["mid"]

#     df = df.set_index("time")

#     roll = df["signal"].rolling(window=window, closed="left")

#     # ➊ maga a szórás
#     df["thr"] = roll.std(ddof=0)

#     # ➋ bal szélek (int64 ns); ha nincs elég adat → NaN
#     left_edge_ns = roll.apply(lambda x: x.index[0].value, raw=False).to_numpy()
#     idx_ns       = df.index.view("int64")                        # int64 ns
#     full_ns      = pd.Timedelta(window).value                   # 24h ⇒ 86.4e12

#     # ➌ igaz, ha van bal szél ÉS az ablak < 24h
#     mask_young = (~np.isnan(left_edge_ns)) & ((idx_ns - left_edge_ns) < full_ns)
#     df.loc[mask_young, "thr"] = np.nan                          # túl fiatal → NaN

#     df = df.reset_index()

#     pos = 0              # 1 long, −1 short, 0 flat
#     entry = None
#     cum_ret = 1.0
#     trades = 0
#     log = []

#     for r in df.itertuples(index=False):
#         thr = r.thr
#         if np.isnan(thr) or thr == 0:
#             continue
#         sig, price, ts = r.signal, r.mid, r.time

#         if pos == 0:
#             if   sig >  thr:
#                 pos, entry, trades =  1, price, trades+1
#                 log.append((ts, "enter_long",  price, np.nan))
#             elif sig < -thr:
#                 pos, entry, trades = -1, price, trades+1
#                 log.append((ts, "enter_short", price, np.nan))

#         elif pos == 1 and sig < -thr:
#             ret = (price - entry) / entry
#             cum_ret *= 1 + ret;  trades += 2
#             log.extend([(ts, "exit_long",  price, ret),
#                         (ts, "enter_short", price, np.nan)])
#             pos, entry = -1, price

#         elif pos == -1 and sig >  thr:
#             ret = (price - entry) / entry
#             cum_ret *= 1 + ret;  trades += 2
#             log.extend([(ts, "exit_short", price, ret),
#                         (ts, "enter_long",  price, np.nan)])
#             pos, entry = 1, price

#     trade_log = (
#         pd.DataFrame(log, columns=["time","action","price","ret"])
#         if return_trade_log else None
#     )
#     return cum_ret, trades, trade_log


# # ──────────────────────────────────────────────────────────────
# # 3)  Wrapper:  időszelet + minden egyben  (pandas df)
# # ──────────────────────────────────────────────────────────────
# def run_pnl_daily(
#     df_book: pd.DataFrame,
#     start: str | dt.datetime,
#     end:   str | dt.datetime,
#     window: str = "24h",          # nézett múlt
#     step:   str = "1D",           # naponta lépünk
#     N: int = 20,
# ):
#     to_dt = lambda x: x if isinstance(x, dt.datetime) else dt.datetime.fromisoformat(x)
#     t_trade0, t_end = map(to_dt, (start, end))
#     w_td, step_td = pd.Timedelta(window), pd.Timedelta(step)

#     rows = []
#     while t_trade0 < t_end:
#         t_lb_start = t_trade0 - w_td          # look-back ablak eleje
#         t_chunk_end = min(t_trade0 + step_td, t_end)

#         sl = df_book[(df_book["received_time"] >= t_lb_start) &
#                      (df_book["received_time"] <  t_chunk_end)]
#         if sl.empty:
#             print(f"[{t_trade0:%Y-%m-%d}]   no data → skip")
#             t_trade0 += step_td
#             continue

#         bench = sl[sl["received_time"] >= t_trade0]
#         if bench.empty:
#             print(f"[{t_trade0:%Y-%m-%d}]   no trade-ticks → skip")
#             t_trade0 += step_td
#             continue

#         # ── benchmark
#         ref_ret = bench["bid_0_price"].iloc[-1] / bench["bid_0_price"].iloc[0]

#         # ── stratégia
#         fm_full = prepare_fair_mid(sl, N=N, time_col="received_time")
#         strat_ret, n_tr, _ = trading_pnl_realtime(
#             fm_full, window=window, return_trade_log=False
#         )
#         # ha nincs pozíció­váltás a napon, strat_ret 1×
#         strat_ret = strat_ret if n_tr > 0 else 1.0

#         rows.append({
#             "trade_date" : t_trade0.date(),
#             "rows_total" : len(sl),
#             "rows_trade" : len(bench),
#             "strat_ret"  : strat_ret,
#             "n_trades"   : n_tr,
#             "bench_ret"  : ref_ret,
#         })

#         print(
#             f"[{t_trade0:%Y-%m-%d}] rows_total={len(sl):,}, "
#             f"rows_trade={len(bench):,}, trades={n_tr}, "
#             f"strat={strat_ret:.4f}×  bench={ref_ret:.4f}×"
#         )

#         t_trade0 += step_td         # következő nap

#     res = pd.DataFrame(rows)

#     g_strat = np.exp(np.log(res["strat_ret"]).mean()) if not res.empty else np.nan
#     g_bench = np.exp(np.log(res["bench_ret"]).mean()) if not res.empty else np.nan
#     return res, g_strat, g_bench




In [8]:
# daily, g_s, g_b = run_pnl_daily(
#     df_book,
#     start="2025-02-05",
#     end="2025-02-10",
#     window="24h",
#     step="3D",
#     N=20,
# )
# print("\nNapi eredmények:\n", daily)
# print(f"\nGeo-átlag strat:  {g_s:.4f}×")
# print(f"Geo-átlag bench:  {g_b:.4f}×")


### cudf-el hosszabb időszakokra

In [9]:
import cudf, cupy as cp, pandas as pd, numpy as np, datetime as dt

def prepare_fair_mid_cudf(gdf: cudf.DataFrame,
                          N: int = 20,
                          batch: int | None = None) -> pd.DataFrame:
    price_cols = [f"{side}_{i}_price" for side in ("bid", "ask") for i in range(N)]
    size_cols  = [f"{side}_{i}_size"  for side in ("bid", "ask") for i in range(N)]
    keep_cols  = ["received_time"] + price_cols + size_cols

    # 1) felesleges oszlopok eldobása (felszabadul a memória)
    gdf = gdf[keep_cols]

    # 2) oszloponkénti in-place(!) downcast
    for col in price_cols + size_cols:
        gdf[col] = gdf[col].astype("float32")          # egyesével → alacsony memória-csúcs
        cp.get_default_memory_pool().free_all_blocks() # rögtön takarítunk

    # 3) fair-mid számítás chunk-ban (ha kérted), egyébként egészben
    def _calc(frame):
        vamp  = cudf.Series(0., index=frame.index, dtype="float32")
        denom = cudf.Series(0., index=frame.index, dtype="float32")
        for p, s in zip(price_cols, size_cols):
            w = frame[p] * frame[s]
            vamp  += w
            denom += frame[s]
        vamp = vamp / denom
        return cudf.DataFrame({
            "time": frame["received_time"],
            "fair": vamp,
            "mid":  (frame["bid_0_price"] + frame["ask_0_price"]) / 2,
        })

    if batch:
        pieces = []
        for i in range(0, len(gdf), batch):
            pieces.append(_calc(gdf.iloc[i:i+batch]))
            cp.get_default_memory_pool().free_all_blocks()
        out = cudf.concat(pieces, ignore_index=True)
    else:
        out = _calc(gdf)

    return out.to_pandas()




# ──────────────────────────────────────────────────────────────
# 2)  gördülő-szórású PnL  (24 h ablak, NaT/NaN diagnosztika)
# ──────────────────────────────────────────────────────────────

def trading_pnl_realtime(
    fm_cu: "cudf.DataFrame | pd.DataFrame",
    window: str = "24h",
    return_trade_log: bool = False,
):
    """
    • Bemenet: cudf DataFrame 3 oszloppal  ['time','fair','mid']
    • A rolling σ-t pandas kezeli   (cuDF még nem támogatja a time-windowt)
    • Vissza: kumulatív hozam, trade-szám, opcionális log  (pandas DF)
    """
    # --- 0) cuDF → pandas (csak a 3 oszlop) ------------------------------
    if isinstance(fm_cu, cudf.DataFrame):
        fm = fm_cu.to_pandas()
    elif isinstance(fm_cu, pd.DataFrame):
        fm = fm_cu.copy()
    else:
        raise TypeError("fm_cu must be cudf or pandas DataFrame")

    orig_len = len(fm)

    # 1) timestamp & NaN szűrés ------------------------------------------
    fm["time"] = pd.to_datetime(fm["time"], errors="coerce")
    fm = fm.dropna(subset=["time", "fair", "mid"])

    print(f"✓ Valid rows for PnL: {len(fm):,} / {orig_len:,}")
    if fm.empty:
        return 1.0, 0, pd.DataFrame() if return_trade_log else None

    # 2) signal + rolling σ (populációs) ---------------------------------
    fm = fm.sort_values("time")
    fm["signal"] = fm["fair"] - fm["mid"]
    fm = fm.set_index("time")

    roll = fm["signal"].rolling(window=window, closed="left")
    fm["thr"] = roll.std(ddof=0)          # szórás az ablakra

    # ablak kor-ellenőrzés (ne legyen < window hossz)
    left_edge_ns = roll.apply(lambda x: x.index[0].value, raw=False).to_numpy()
    now_ns       = fm.index.view("int64")
    window_ns    = pd.Timedelta(window).value
    young = (~np.isnan(left_edge_ns)) & ((now_ns - left_edge_ns) < window_ns)
    fm.loc[young, "thr"] = np.nan         # fiatal ablak → nincs trade

    fm = fm.reset_index()

    # 3) pozíciókezelés ---------------------------------------------------
    pos = 0; entry = None; cum_ret = 1.0; trades = 0; log = []
    for r in fm.itertuples(index=False):
        thr = r.thr
        if np.isnan(thr) or thr == 0:
            continue                # nincs küszöb ⇒ nincs trade

        sig, price, ts = r.signal, r.mid, r.time

        if pos == 0:                # flat → belépés
            if   sig >  thr:
                pos, entry, trades =  1, price, trades+1
                log.append((ts,"enter_long", price, np.nan))
            elif sig < -thr:
                pos, entry, trades = -1, price, trades+1
                log.append((ts,"enter_short", price, np.nan))

        elif pos == 1 and sig < -thr:      # long zár, short nyit
            ret = (price-entry)/entry
            cum_ret *= 1+ret; trades += 2
            log.extend([(ts,"exit_long", price, ret),
                        (ts,"enter_short", price, np.nan)])
            pos, entry = -1, price

        elif pos == -1 and sig >  thr:     # short zár, long nyit
            ret = (price-entry)/entry
            cum_ret *= 1+ret; trades += 2
            log.extend([(ts,"exit_short", price, ret),
                        (ts,"enter_long",  price, np.nan)])
            pos, entry = 1, price

    trade_log = (
        pd.DataFrame(log, columns=["time","action","price","ret"])
        if return_trade_log else None
    )
    return cum_ret, trades, trade_log

# ──────────────────────────────────────────────────────────────
# 3)  Wrapper:  időszelet + minden egyben  (pandas df)
# ──────────────────────────────────────────────────────────────
def run_pnl(
    df_book: pd.DataFrame,
    start: str | dt.datetime | None = None,
    end:   str | dt.datetime | None = None,
    window: str = "24h",
    N: int = 20,
    return_trade_log: bool = True,
):

    # --- 0) időintervallum szűrés -------------------------------------
    parse = lambda x: None if x is None else (
        dt.datetime.fromisoformat(x) if isinstance(x, str) else x
    )
    s_dt, e_dt = parse(start), parse(end)

    if s_dt: df_book = df_book[df_book["received_time"] >= s_dt]
    if e_dt: df_book = df_book[df_book["received_time"] <= e_dt]

    # --- 1) kereskedési & benchmark periódus azonosítása --------------
    win_td          = pd.Timedelta(window)
    first_ts_total  = df_book["received_time"].min()
    trading_start   = (s_dt if s_dt else first_ts_total) + win_td
    trading_end     = e_dt if e_dt else df_book["received_time"].max()

    # --- 2) benchmark (bid₀ ár) számítása -----------------------------
    bench_slice = df_book[df_book["received_time"] >= trading_start]

    if not bench_slice.empty:
        first_price = bench_slice["bid_0_price"].iloc[0]
        last_price  = bench_slice["bid_0_price"].iloc[-1]
        ref_ret     = last_price / first_price
        print(f"▶ Benchmark időszak : {trading_start}  →  {trading_end}")
        print(f"▶ Benchmark return  : {ref_ret:.4f}× (bid₀ ár)")
    else:
        ref_ret = np.nan
        print(f"▶ Benchmark időszak : {trading_start}  →  {trading_end}  |  NINCS adat")

    # --- 3) kereskedési periódus kiírása ------------------------------
    print(f"▶ Kereskedési időszak: {trading_start}  →  {trading_end}")

    # --- 4) stratégia futtatása ---------------------------------------
    fm = prepare_fair_mid_cudf(df_book, N=N)
    cum_ret, trades, log = trading_pnl_realtime(
        fm, window=window, return_trade_log=return_trade_log
    )

    print(f"Hozam: {cum_ret:.4f}× | Trades: {trades}")
    return cum_ret, trades, log

In [ ]:
cum_ret, n_trades, trade_log = run_pnl(
    df_book,                       # pandas DataFrame (Lake 'book')
    start="2024-09-01 00:00:00",
    end  ="2024-09-15 00:00:00",
    window="24h",           
    N=10,
)

▶ Benchmark időszak : 2024-09-02 00:00:00  →  2024-09-15 00:00:00
▶ Benchmark return  : 1.0470× (bid₀ ár)
▶ Kereskedési időszak: 2024-09-02 00:00:00  →  2024-09-15 00:00:00


RuntimeError: parallel_for: failed to synchronize: cudaErrorMemoryAllocation: out of memory